In [150]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import Delaunay
from sklearn.metrics.cluster import fowlkes_mallows_score
import vg
import plotly.graph_objects as go
import ray
from pytransform3d.rotations import matrix_from_axis_angle
from scipy.spatial.transform import Rotation as Rot
from tqdm import tqdm
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
import time

import warnings
warnings.filterwarnings('ignore')

def go_cluster(cluster):
    
    if(len(cluster.galaxies)==1):
        return [go.Scatter3d(
    x=cluster.galaxies[:,0],
    y=cluster.galaxies[:,1],
    z=cluster.galaxies[:,2],
    mode='markers',
    marker=dict(
        size=1,
        opacity=1,
        color='green'
    )
    )]
    
    return [go.Scatter3d(
    x=cluster.galaxies[:,0],
    y=cluster.galaxies[:,1],
    z=cluster.galaxies[:,2],
    mode='markers',
    marker=dict(
        size=3,
        opacity=1,
        color='green'

    )
    )]+[cluster.get_mesh()]+[go.Scatter3d(
    x=[cluster.centroid[0]],
    y=[cluster.centroid[1]],
    z=[cluster.centroid[2]],
    mode='markers',
    marker=dict(
        size=2,
        opacity=1,
        color='red'
    )
    )]

def get_clusters_plot(clusters,is_show=False,filename='3d_model'):
    
    data=[]
    for cluster in clusters:
        data+=go_cluster(cluster)
    
    fig = go.Figure(data=data)

# tight layout
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    if(is_show):
        fig.show()
    fig.write_html(filename+".html")


In [204]:
coor=pd.read_csv('ra_dec_z_data.csv')
coor_with_labels=pd.read_csv('coor_with_labels.csv')

In [212]:
tmp_coor_with_labels=coor_with_labels[['x','y','z','label']]
tmp_coor_with_labels=tmp_coor_with_labels[tmp_coor_with_labels.x.between(40, 120)]
tmp_coor_with_labels=tmp_coor_with_labels[tmp_coor_with_labels.y.between(-40, 40)]
tmp_coor_with_labels=tmp_coor_with_labels[tmp_coor_with_labels.z.between(-40, 40)]

tmp_coor_with_labels.shape

(1341, 4)

In [213]:
tmp_coor=coor[['x','y','z']]
tmp_coor=tmp_coor[tmp_coor.x.between(40, 80)]
tmp_coor=tmp_coor[tmp_coor.y.between(-20, 20)]
tmp_coor=tmp_coor[tmp_coor.z.between(-20, 20)]
tmp_coor.shape

(1307, 3)

In [4]:
# fig = go.Figure(data=[go.Scatter3d(
#     x=tmp_coor.x,
#     y=tmp_coor.y,
#     z=tmp_coor.z,
#     mode='markers',
#     marker=dict(
#         size=1,
#         opacity=1,
#         color='red'
#     )
#     )])

# # tight layout
# fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
# fig.show()

In [198]:
np.sort(np.concatenate([np.array([1,9,3]),[4]]))[:,1]

IndexError: too many indices for array

In [226]:
class Cluster():
    # non shifted shape, easy to expand
    non_rotated_cube=(np.transpose(np.array([[0, 0, 1, 1, 0, 0, 1, 1],[0, 1, 1, 0, 0, 1, 1, 0],[0, 0, 0, 0, 3, 3, 3, 3]]))-np.array([0.5,0.5,1.5]))*0.05
    rotated_cube=None# shifted coor of rotated cube
    centroid=None# coor of center
    galaxies=None

    is_complete=False
    times_grow=0
    def __init__(self, center,non_rotated_cube=None,galaxies=None):
        
        if(isinstance(non_rotated_cube, np.ndarray)):
            self.non_rotated_cube=non_rotated_cube
        if(not isinstance(galaxies, np.ndarray)):
            self.galaxies=np.array([center])
        else:
            self.galaxies=np.array(galaxies)
        
        self.centroid=center[:3]
        self.rotated_cube = self.rotate(self.centroid,self.non_rotated_cube)+self.centroid
        
        for gal in self.galaxies:
            if(Delaunay(self.rotated_cube).find_simplex(gal[:3]) < 0):
                raise ValueError
        
        
    def rotate(self, vector, points):    
        vector = vg.normalize(vector)
        axis = vg.perpendicular(vg.basis.z, vector)
        angle = vg.angle(vg.basis.z, vector, units='rad')
        
        a = np.hstack((axis, (angle,)))
        R = matrix_from_axis_angle(a)
        
        r = Rot.from_matrix(R)
        rotmat = r.apply(points)
        
        return rotmat

    def get_mesh(self):
        return go.Mesh3d(
        x=self.rotated_cube[:,0],
        y=self.rotated_cube[:,1],
        z=self.rotated_cube[:,2],
        opacity=0.5,
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],color='blue'
        )
    
    def grow(self,coef):
        
#         if((self.non_rotated_cube[1]-self.non_rotated_cube[0])[1]*3>(self.non_rotated_cube[4]-self.non_rotated_cube[0])[2]):
#             coef=np.array([coef,coef,3*coef])
        if(self.times_grow==5):
            self.is_complete=True
            return
        self.times_grow+=1
        self.non_rotated_cube=self.non_rotated_cube*coef
        self.rotated_cube = self.rotate(self.centroid,self.non_rotated_cube)+self.centroid            
    
class Clusterer:
    
    unit_cube=(np.transpose(np.array([[0, 0, 1, 1, 0, 0, 1, 1],[0, 1, 1, 0, 0, 1, 1, 0],[0, 0, 0, 0, 1, 1, 1, 1]]))-0.5)
    
    def __init__(self):
        self.clusters=[]
        ray.init(log_to_driver=False)
    
    def merge(self):
        
        clusters_len=len(self.clusters)
        #graph=np.zeros((clusters_len,clusters_len))
        #ray.shutdown()

        clusters_id = ray.put(self.clusters)

        graph=ray.get([self.parallel_connectivity_matrix.remote(self,clusters_id, subset) for subset in self.split_array(np.arange(len(self.clusters)),4)])
        #graph=ray.get([self.parallel_connectivity_matrix.remote(self,clusters_id, row) for row in range(len(self.clusters))])

        graph = np.concatenate( graph, axis=0 )
#         for i in tqdm(range(clusters_len),position=0, leave=True):
#             for j in range(clusters_len):
#                 if(np.linalg.norm(self.clusters[i].centroid-self.clusters[j].centroid)>3):
#                     continue
#                 if(self.check_collision(self.clusters[i],self.clusters[j])):
#                     graph[i,j]=1
        graph = graph+np.transpose(graph)
        graph=graph-np.eye(len(self.clusters))

        graph = csr_matrix(graph)
        n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
        
        components=[]
        
        np_clusters=np.array(self.clusters)
        
        for label in np.unique(labels):
            components.append(np_clusters[labels==label])
        
        new_clusters=[]
 
        for component in components:
            if(len(component)==1):
                new_clusters.append(component[0])
            else:
                new_clusters.append(self.collide_clusters(component))
        
        self.clusters=new_clusters        
      

    def check_collision(self,a:Cluster, b: Cluster):      
        i=a.rotated_cube[1]-a.rotated_cube[0]
        j=a.rotated_cube[3]-a.rotated_cube[0]
        k=a.rotated_cube[4]-a.rotated_cube[0]
        for point in b.rotated_cube:
            u=point-a.rotated_cube[0]
            if((0<= np.dot(u,i))and(np.dot(u,i) <= np.dot(i,i))and
               (0<= np.dot(u,j))and(np.dot(u,j) <= np.dot(j,j))and
               (0<= np.dot(u,k))and(np.dot(u,k) <= np.dot(k,k))):
                return True
            
        i=b.rotated_cube[1]-b.rotated_cube[0]
        j=b.rotated_cube[3]-b.rotated_cube[0]
        k=b.rotated_cube[4]-b.rotated_cube[0]
        for point in a.rotated_cube:
            u=point-b.rotated_cube[0]
            if((0<= np.dot(u,i))and(np.dot(u,i) <= np.dot(i,i))and
               (0<= np.dot(u,j))and(np.dot(u,j) <= np.dot(j,j))and
               (0<= np.dot(u,k))and(np.dot(u,k) <= np.dot(k,k))):
                return True
        return False
 
 #         for point in b.rotated_cube:
#             if(Delaunay(a.rotated_cube).find_simplex(point) >= 0):
#                 return True   
#         return False
    
    def rotation_matrix_from_vectors(self, vec2):
        vec1=[1,0,0]
        a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
        v = np.cross(a, b)
        c = np.dot(a, b)
        s = np.linalg.norm(v)
        kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
        return rotation_matrix
    
    def collide_clusters(self, clusters:[Cluster]):
        
        if(len(clusters)==1):
            return clusters[0]
        
        galaxies=[]
        vertex_points=[]
        for cluster in clusters:
            galaxies.append(cluster.galaxies)
            vertex_points.append(cluster.rotated_cube)
        galaxies=np.concatenate( galaxies, axis=0 )
        vertex_points=np.concatenate( vertex_points, axis=0 )
        
        center = galaxies[:,:3].mean(axis=0)

        projections = center * np.dot(galaxies[:,:3], np.transpose([center])) / np.dot(center, center)
        
        distances_on_line=np.linalg.norm(projections-center,axis=1)
        vectors_from_line=galaxies[:,:3]-projections
    
        length = distances_on_line.max()
        width=np.linalg.norm(vectors_from_line,axis=1).max()
    
        cube=self.unit_cube.copy()*[width*2+0.05,width*2+0.05,length*2+0.05]
    
        return Cluster(center,cube,galaxies)
            
    def compress_cluster(self,cluster):

        galaxies=cluster.galaxies
        if(galaxies.ndim==1 ):
            galaxies=np.array([galaxies])
        vertex_points=cluster.rotated_cube
        
        center = galaxies[:,:3].mean(axis=0)

        projections = center * np.dot(galaxies[:,:3], np.transpose([center])) / np.dot(center, center)
        
        distances_on_line=np.linalg.norm(projections-center,axis=1)
        vectors_from_line=galaxies[:,:3]-projections
    
        length = distances_on_line.max()
        width=np.linalg.norm(vectors_from_line,axis=1).max()
    
        cube=self.unit_cube.copy()*[width*2+0.05,width*2+0.05,length*2+0.05]
    
        return Cluster(center,cube,galaxies)
    
    def step(self,grow_coef):
        
        is_done=True
        for cluster in self.clusters:
            if(not cluster.is_complete):
                is_done=False
                break
        if (is_done):
            return False
        
        for cluster in self.clusters:
            cluster.grow(grow_coef)
        self.merge()
        return True
      
    def split_array(self,a, n):
        k, m = divmod(len(a), n)
        return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))
    
    @ray.remote
    def parallel_connectivity_matrix(self,clusters ,rows):
        new_graph=np.empty((len(rows),len(clusters)))
        for i in range(len(rows)):
            for j in range(rows[i],len(clusters)):
                if(np.linalg.norm(clusters[rows[i]].centroid-clusters[j].centroid)>3):
                    continue
                if(self.check_collision(clusters[rows[i]],clusters[j])):
                    new_graph[i,j]=1
        return new_graph
    
    def fit(self,data):
        coefs=1+1/np.arange(1,100)[10:]*5
        data_np=np.array(data)
        for i in range(len(data_np)):
            self.clusters.append(Cluster(np.concatenate([data_np[i],[i]])))
        iter_num=1
        start = time.time()
        while(self.step(coefs[iter_num-1])):
            print('iter : ',iter_num,', n_clusters: ', len(self.clusters),', time: ',time.time()-start,' s')
            iter_num+=1
            start = time.time()
        
        galaxies=[]
        for i in range(len(self.clusters)):
            cluster_galaxies=self.clusters[i].galaxies
#             if(len(cluster_galaxies)==1):
#                 cluster_galaxies[:,;-1]=-1
#             else:
            
            cluster_galaxies =np.append(cluster_galaxies, np.ones((len(cluster_galaxies),1))*i, axis=1)

            galaxies.append(cluster_galaxies)
           
        clusters=[]
        for cluster in self.clusters:
            clusters.append(self.compress_cluster(cluster))
            
        self.clusters=clusters
        galaxies=np.concatenate(galaxies)
        galaxies = galaxies[galaxies[:,3].argsort()]  
        return galaxies[:,4]
        

In [227]:
%%time
clusterer=Clusterer()
labels=clusterer.fit(tmp_coor_with_labels[['x','y','z']])
print(fowlkes_mallows_score(tmp_coor_with_labels['label'],labels))
ray.shutdown()
get_clusters_plot(clusterer.clusters)

2021-05-07 02:35:19,075	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265


iter :  1 , n_clusters:  1333 , time:  6.698239803314209  s
iter :  2 , n_clusters:  1323 , time:  6.359395265579224  s
iter :  3 , n_clusters:  1299 , time:  6.47526216506958  s
iter :  4 , n_clusters:  1249 , time:  7.617640256881714  s
iter :  5 , n_clusters:  1193 , time:  6.23497748374939  s
iter :  6 , n_clusters:  1180 , time:  5.476889371871948  s
iter :  7 , n_clusters:  1170 , time:  5.271589756011963  s
iter :  8 , n_clusters:  1150 , time:  7.01472020149231  s
iter :  9 , n_clusters:  1115 , time:  5.620779991149902  s
iter :  10 , n_clusters:  1083 , time:  5.342155694961548  s
iter :  11 , n_clusters:  1047 , time:  4.870005369186401  s
iter :  12 , n_clusters:  1023 , time:  4.362858057022095  s
iter :  13 , n_clusters:  993 , time:  4.302340507507324  s
iter :  14 , n_clusters:  965 , time:  3.9373340606689453  s
iter :  15 , n_clusters:  945 , time:  4.048976898193359  s
iter :  16 , n_clusters:  927 , time:  5.716234445571899  s
iter :  17 , n_clusters:  917 , time:  

In [225]:
ray.shutdown()


In [239]:
import hdbscan
model=hdbscan.HDBSCAN().fit(tmp_coor_with_labels[['x','y','z']])
labels =model.labels_
c=1
for i in range(len(labels)):
    if(labels[i]==-1):
        labels[i]=400000+c
        c+=1
print(fowlkes_mallows_score(tmp_coor_with_labels['label'],labels))

0.3328129784353538


In [228]:
tmp_coor_with_labels['label'].value_counts().value_counts()

1     535
2      98
3      32
4      15
5       9
6       8
7       7
8       5
10      3
19      2
67      1
34      1
12      1
16      1
17      1
20      1
29      1
9       1
Name: label, dtype: int64

In [230]:
pd.Series(labels).value_counts().value_counts()

1     818
2      54
3       6
20      3
11      2
4       2
5       2
9       2
14      2
16      2
18      2
38      1
6       1
21      1
22      1
27      1
69      1
dtype: int64

In [64]:
step_coef=[1.6,1.4,1.2,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.05,1.05,1.05,1.05,1.05]

for coef in tqdm(step_coef,position=0, leave=True):
    clusterer.step(coef)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [02:25<00:00,  9.70s/it]


In [66]:
get_clusters_plot(clusterer.clusters)

In [10]:

100%|██████████████████████████████████████████████████████████████████████████████| 1307/1307 [02:09<00:00, 10.06it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1192/1192 [01:44<00:00, 11.38it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1152/1152 [01:37<00:00, 11.87it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1092/1092 [01:18<00:00, 13.94it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1050/1050 [01:10<00:00, 14.79it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1022/1022 [01:04<00:00, 15.88it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 987/987 [00:59<00:00, 16.51it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 963/963 [00:54<00:00, 17.52it/s]

SyntaxError: invalid syntax (<ipython-input-10-c20a188cfc06>, line 1)